In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(33, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    author_id = row.author_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    parameters = {
        # 'candidate': [2, 3, 4, 5],
        # 'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        'samples': [50, 100, 150, 200, 250, 300, 400, 600, 1000, 1500, 2000, 3000, 4000],
        # 'dimensions': [50, 100, 200],
        # 'iterations': [40, 80, 120, 160, 200],
    }
    
    print('doc_id: %3s actual author: %4s author_id: %s' % (str(doc_id), str(row.author_id), str(authorList)))
    
    level = 'word'
    
    for idxp, paralist in parameters.iteritems():
        
        if idxp == 'samples':
            for idxl, val in enumerate(paralist):
                
                doc_id = doc_id
                candidate = len(authorList)
                dimensions = 200
                samples = val
                iterations = 80
                dropout = 0.4
                test = idxp
                
                printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
                printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
                printstate += (("\niterations = %s, dropout = %s, ") % (str(iterations), str(dropout)))
                printstate += (("test = %s") % (str(test)))
                
                print("Current test: %s" % (str(printstate)))
                
                import UpdateDB as db
                
                case = db.checkOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                   samples = samples, iterations = iterations, dropout = dropout, test = test)
                
                if case == False:
                    print("Running: %12s" % (str(printstate)))
                    
                    import StyloNeural as Stylo
                    (labels_index, predYList, predY, history, samples) = Stylo.getResults(
                        doc_id = doc_id, authorList = authorList[:(candidate + 1)], 
                        level = level, glove = '../../glove/', dimensions = dimensions, 
                        samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
                    
                    for key, auth in labels_index.iteritems():
                        if auth == author_id:
                            loc = key
                    
                    import UpdateDB as db
                    case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                              samples = samples, iterations = iterations, dropout = dropout,
                                              accuracy = predY[loc], test = test)
                    
                    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
                    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
                    printstate += (("\niterations = %s, dropout = %s, ") % (str(iterations), str(dropout)))
                    printstate += (("test = %s") % (str(test)))
                    
                    if case == False:
                        print("Completed: %12s" % (str(printstate)))

                    else:
                        print("Skipped: %12s" % (str(printstate)))

                    from IPython.display import clear_output
                    clear_output()

                else:
                    print("Skipped: %12s" % (str(printstate)))
                
            

Current test: doc_id = 85, candidate = 3, dimensions = 200, samples = 200, 
iterations = 80, dropout = 0.4, test = samples
Execution completed
Running: doc_id = 85, candidate = 3, dimensions = 200, samples = 200, 
iterations = 80, dropout = 0.4, test = samples
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 124
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (11578, 2)
Author:    44  Size:  4746
Author:    64  Size:  5106
Author:    82  Size:  1726
Min: 1726
Max: 5106
Authors [44, 64, 82].
Found 600 texts.
Found 600 labels.
Found 21694 unique tokens.
Shape of data tensor: (600, 1000)
Shape of label tensor: (600, 3)
Done compiling.
Train on 540 samples, validate on 60 samples
Epoch 1/80
540/540 [==============================] - 3s - loss: 1.1038 - acc: 0.3296 - val_loss: 1.0984 - val_acc: 0.3500

In [ ]:
%tb